In [9]:
import sys,os 
sys.path.append('/home/benr/ACT/CW2/py')
import numpy as np 


import pandas as pd 
from PIL import Image, UnidentifiedImageError
from sklearn.preprocessing import LabelEncoder
from functions import get_data, galaxy_type,split_data_torch,train
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [10]:
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("CUDA_VISIBLE_DEVICES:", os.environ.get("CUDA_VISIBLE_DEVICES"))

CUDA available: True
CUDA version: 12.8
CUDA_VISIBLE_DEVICES: None


# Question 2, Convolutional Neural Network for image recognition 
In this section the aim is to repeat the same process as question 1 but replace the traditional reandom forest method with a convolutional neural network. The expectation is that due to the CNN's ability to correlate spacial features, it will be a more roubust method of recognising galaxy morphology.

The first step will be to load the data in exactly the same way as Q1. 

In [ ]:

# get the data from functions.py
images,labels = get_data()
# convert images to pytorch tnesors
images = torch.from_numpy(images)
labels = torch.from_numpy(labels).long()

print(images.shape)


/home/benr/.astroNN/datasets/Galaxy10_DECals.h5 was found!
torch.Size([17736, 256, 256, 3])


: 

In [ ]:
# Move channels: (N,H,W,C) -> (N,C,H,W)
images = images.permute(0, 3, 1, 2)      

# standardise 
images = images.float() / 255.0      
mean = images.mean(dim=[0,2,3])
std = images.std(dim=[0,2,3])
print(images.shape)
print(mean)
print(std)

For the CNN the lables will need to be encoded into numerical values. 
Documentation can be found here: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

Now we set up the class that will hold the structure of the network. Galaxy_CNN takes the number of input channels, the number of classes (Prediction catagories) and passes them two the different layers of the network. 
Throughout the process of data being passed through different levels of the CNN we will need to keep trach of the network is change the size of each tensor. This can be done with the following formulas

* CNN Output - $$\frac{H + 2P -K}{s} + 1 $$
* Pooling Output - $$\frac{H-K}{s}  + 1$$

H = input height of width of image 


K = kernel size

P = padding size

s = stride 

In [ ]:

class Galaxy_CNN(nn.Module):
    def __init__(self, inCH, nCL):  
        super(Galaxy_CNN, self).__init__()
        # conv blocks
        self.conv1 = nn.Conv2d(3, inCH, kernel_size=5, stride=1, padding=4)
        self.bn1   = nn.BatchNorm2d(inCH)

        self.conv2 = nn.Conv2d(inCH, inCH*2, kernel_size=3, stride=1, padding=1)
        self.bn2   = nn.BatchNorm2d(inCH*2)

        self.conv3 = nn.Conv2d(inCH*2, inCH*2, kernel_size=3, stride=1, padding=1)
        self.bn3   = nn.BatchNorm2d(inCH*2)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.35)

        # global average pooling 
        self.gap = nn.AdaptiveAvgPool2d(1)   
        self.lin = nn.Linear(inCH*2, nCL)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.dropout(x)

        x = self.gap(x)              
        x = x.view(x.size(0), -1)    
        x = self.lin(x)
        return x

In [ ]:
#split tensor into training and test sets (see functions.py)
I = np.arange(len(labels))
xtrn,xtst,ytrn,ytst = split_data_torch(images,labels,I)



train_ds = TensorDataset(xtrn, ytrn)
test_ds  = TensorDataset(xtst, ytst)




train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=8, shuffle=False)


In [ ]:
num_classes = len(labels.unique())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Galaxy_CNN(128,num_classes).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=20**-4,weight_decay=0.0001)

#for normalisation 
mean_tensor = mean.view(1,3,1,1).to(device)
std_tensor = std.view(1,3,1,1).to(device)


print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))


0
NVIDIA GeForce RTX 4060 Laptop GPU


In [ ]:
train(model, train_loader,test_loader,  
      mean_tensor,std_tensor,
      criterion, 
      optimizer, device, 
      n_epoch=100)


Epoch 1/100 Train loss: 2.2669  Train acc: 0.200  Test acc: 0.248


KeyboardInterrupt: 